# Multirotor Altitutde (Height) Control Mission

This lab demonstartes cascade PI controllers in Python. The mission of the lab is straight forward: maintaining altitude at 20 meters. To acheive it, we will implement 2 PI controllers. One for controlling velocity of the multirotor and another for controlling position (altitude).

By default, the multirotor in Ubicoders virtual robot system loads wihtout any system or sensor noise. We will use this default mode for the sake of demonstrating the casecase controller itself. If you wish to turn on the noise, the system noise can be turned on from menu(esc) -> settings -> system noise and/or sensor noise.

## 1. Getting Started

This entire environment assumes your local machine at the moment. Therefore, please have

- 1. Python 3.9+
- 2. pip install ubicoders-vrobots, ipython, numpy, matplotlib, cv2
  
Optionally, if you are using VSCode,

- 3. Python extention for VSCode by Microsoft

## 1. Start the bridge servers.

Run the cell below and check the virtual robot webpage.  The green light means it is ready to control the virtual robot!

In [1]:
import threading
import asyncio
from ubicoders_vrobots import run_servers
from ubicoders_vrobots.vrobots_clients.vr_client_utils import (
    VirtualRobot,
    vr_client_main,
)
from ubicoders_vrobots.vrobots_msgs.python.states_msg_helper import VRobotState

def run_bridge():
    try:
        asyncio.run(run_servers())
    except KeyboardInterrupt:
        print("Server stopped by user.")

try:
    script_thread = threading.Thread(target=run_bridge)
    script_thread.start()
finally:
    print("Ignition finished")

Ignition finished


Virtual Robots REST Bridge is running @ port 12741
Virtual Robots Bridge is running @ port 12740


In [13]:
class MyVRobotsController:
    def __init__(self, vr: VirtualRobot = None):
        self.err_int_vel = 0
        self.vr = vr

    def vel_err_int(self, vel_err):
        self.err_int_vel += vel_err
        if self.err_int_vel > 100:
            self.err_int_vel = 100
        elif self.err_int_vel < -100:
            self.err_int_vel = -100
        return self.err_int_vel

    def vel_ctrl(self, sp_vel, vel):
        error = sp_vel - vel
        throttle = error * 10 + 1200 + self.vel_err_int(error) * 3
        return throttle

    def loop(self):
        states: VRobotState = self.vr.states
        # print(f"states: {states}")
        print(f"system id: {states.sysId}")
        print(f" time: {states.timestamp}")
        print(f" linPos: {states.linPos}")

        altitude = -states.linPos.z
        sp_alt = 20

        error = sp_alt - altitude
        sp_vel = error * 10 + states.linVel.z * 25

        # print(f"sp_vel: {sp_vel}, vel: {states.linVel.z}")

        throttle = self.vel_ctrl(sp_vel, states.linVel.z)

        # vr.update_cmd_omrover([0.0, 0.0, 10.0, 20.0])
        self.vr.update_cmd_multirotor(0, [throttle, throttle, throttle, throttle])
        # vr.update_cmd_car(3000, 0, 0)
        #self.vr.update_cmd_set_force_torque_body(0, 0, 0, -(10 * 2) * 9.82, 0, 0, 0.0)


vr_client_main(MyVRobotsController(), duration=15)


Opened connection
system id: 0
 time: 0.0
 linPos: {
    "x": 0,
    "y": 0,
    "z": 0
}
system id: 0
 time: 1719798244206.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.17302900552749634
}
system id: 0
 time: 1719798244235.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.17302899062633514
}
system id: 0
 time: 1719798244252.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.18323135375976562
}
system id: 0
 time: 1719798244269.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.20363610982894897
}
system id: 0
 time: 1719798244269.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.20363610982894897
}
system id: 0
 time: 1719798244328.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.27505284547805786
}
system id: 0
 time: 1719798244328.0
 linPos: {
    "x": 50.599998474121094,
    "y": 180